In [1]:
%matplotlib inline

# Notes:

# Imports

In [2]:
import pandas as pd
from Bio import SearchIO

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier

from collections import OrderedDict

import joblib
import random

# Constants

In [3]:
benchmark_file = '../Data/phage_data_nmicro2017/processed_benchmark_set.csv'

hmm_results_dir = '../Data/phage_data_nmicro2017/hmmsearch_out/'

base_dir = '../Data/classifier_data/'

hmm_data_dir = '../Data/protein_domain_data/domain_alignments_and_hmms/'

# Initial data processing

In [4]:
###Read in the dataset and double check that analysis is limited to empirically defined data
df = pd.read_csv(benchmark_file, index_col=0)
print('Starting shape:', df.shape)
df = df[df['Temperate (empirical)'] != 'Unspecified']
print('New shape (should be identical):', df.shape)

###Add in my identifier
df['Identifier_AJH'] = ''  
df.at[df[df['Database source'] == 'NCBI RefSeq'].index, 'Identifier_AJH'] =\
                    df[df['Database source'] == 'NCBI RefSeq']['RefSeq accession number']
df.at[df[df['Database source'] == 'Actinobacteriophage_785'].index, 'Identifier_AJH'] =\
                    df[df['Database source'] == 'Actinobacteriophage_785']['Virus identifier used for the analysis'].str.split('_').str[0]
print('New shape (+1 new column):', df.shape)

Starting shape: (1057, 22)
New shape (should be identical): (1057, 22)
New shape (+1 new column): (1057, 23)


In [5]:
###Read through all of the hmmsearch files to accumulate a growing presence/absence dataframe
file_ending = '.fasta.hmmsearch'
growing_df = pd.DataFrame()
for index in df.index:
    if df.loc[index]['Database source'] == 'NCBI RefSeq':
        file_name = df.loc[index]['RefSeq accession number'] + file_ending
    elif df.loc[index]['Database source'] == 'Actinobacteriophage_785':
        file_name = df.loc[index]['Virus identifier used for the analysis'].split('_')[0].lower() +\
                    file_ending
    try:
        with open(hmm_results_dir + file_name, 'r') as infile:
            results = list(SearchIO.parse(infile, 'hmmer3-text'))
            simple_res = []
            for i in results:
                if len(i.hits) > 0:
                    simple_res.append((i.id, 1))
                else:
                    simple_res.append((i.id, 0))
        simple_res = sorted(simple_res, key=lambda x: x[0])
        single_df = pd.DataFrame(OrderedDict(simple_res), index=[index])
        growing_df = pd.concat([growing_df, single_df])
    except FileNotFoundError:
        print('Failed to find this file: {}!'.format(file_name))
        pass
print(growing_df.shape)

###Add that to the main dataframe
full_df = df.join(growing_df)

(1057, 371)


In [6]:
###Split into training and testing sets
train_df, test_df = train_test_split(full_df, train_size=0.6,\
                                     random_state=42)
print('Shape of training and testing dataframes:', train_df.shape, test_df.shape)

###Set up the machine-learning training and test sets
ml_df_train = train_df[train_df.columns[23:]]
ml_df_test = test_df[test_df.columns[23:]]

###And labels
training_labels = pd.DataFrame(index=train_df.index)
training_labels['binary'] = 0
training_labels.at[train_df[train_df['Temperate (empirical)']=='yes'].index, 'binary'] = 1
training_labels = training_labels['binary']

testing_labels = pd.DataFrame(index=test_df.index)
testing_labels['binary'] = 0
testing_labels.at[test_df[test_df['Temperate (empirical)']=='yes'].index, 'binary'] = 1
testing_labels = testing_labels['binary']

print('Shape of training and testing labels:', training_labels.shape, testing_labels.shape)

Shape of training and testing dataframes: (634, 394) (423, 394)
Shape of training and testing labels: (634,) (423,)


# Drop features that are likely to be noise

My goal in semi-rationally selecting these protein domains was to *predict temperate phages*, ergo I don't even want a predictor of lytic phages in my dataset and given the choice of protein domains that I am including believe that these would likely be noise.

**Note that this step is reasonable if I *only* consider the training set in making this decision, as I am doing**

In [7]:
uninformative_cols = []
###Use training set to remove certain cases with too few hits (likely unreliable/not useful)
too_few_count = 2
transpose_df = ml_df_train.transpose()
uninformative_cols.extend(list(transpose_df[transpose_df.sum(axis=1)<=too_few_count].index))
print('Running count of uninformative columns:', len(uninformative_cols))
###Cases where lytic features are higher than or equal to temperate
lysog_df = ml_df_train[train_df['Temperate (empirical)']=='yes'].transpose()
lytic_df = ml_df_train[train_df['Temperate (empirical)']=='no'].transpose()
uninformative_cols.extend(list(transpose_df[lysog_df.sum(axis=1) <= (lytic_df.sum(axis=1))].index))
uninformative_cols = list(set(uninformative_cols))
print('Running count of uninformative columns:', len(uninformative_cols))

Running count of uninformative columns: 55
Running count of uninformative columns: 165


**Drop these columns from train, test sets**

In [8]:
ml_df_train = ml_df_train.drop(columns=uninformative_cols)
print('Training set shape', ml_df_train.shape)

ml_df_test = ml_df_test.drop(columns=uninformative_cols)
print('Testing set shape', ml_df_test.shape)

Training set shape (634, 206)
Testing set shape (423, 206)


**Write training and testing dataframes to a file**

In [9]:
train_df.drop(columns=uninformative_cols).to_csv(base_dir + 'train_df.csv')
test_df.drop(columns=uninformative_cols).to_csv(base_dir + 'test_df.csv')

**And write a new `.hmm` file containing only these *putatively* useful domains**

In [10]:
identifiers = list(ml_df_train.columns)
with open(base_dir+'best_models.hmm', 'w') as outfile:
    for i in identifiers:
        fname = hmm_data_dir+'{}.hmm'.format(i)
        with open(fname) as infile:
            outfile.write(infile.read())

In [11]:
print(len(identifiers))

206


# Machine learning of a model

This is using a very basic Random Forest classifier with grid search to optimize hyper parameters via cross-validation. I'm using/testing 2 different cross-validation strategies:

    1. normal k-fold cross-validation
        - where best is the highest mean amongst cross-val scores (standard approach)
    2. more intense bootstrap sampling version
        - where best is the highest minimum amongst cross-val scores (my interpretation)

## Traditional cross validation

In [12]:
###Really simple random forest hyper-parameter sweep
scoring_fxn = 'f1'
n_fold_cv = 5
#
rf = RandomForestClassifier()
params_rf = {'bootstrap': [True, False],\
             'class_weight':['balanced', 'balanced_subsample'],\
             'min_samples_leaf': [1, 2],\
             'n_estimators': list(range(10, 105, 5)),\
             'max_depth': list(range(10, 42, 2))}

rf_gs = GridSearchCV(rf, params_rf, scoring=scoring_fxn, cv=n_fold_cv)

#Fit the model
rf_gs.fit(ml_df_train, training_labels)

#Select the best model (this selects the parameter set with the best mean score across cv splits)
rf_best = rf_gs.best_estimator_
print('Parameters of best model:', rf_gs.best_params_)

print('Cross validation scores using that model:',\
      cross_val_score(rf_gs.best_estimator_, ml_df_train, training_labels,\
                cv=n_fold_cv, scoring=scoring_fxn))

joblib.dump(rf_best, base_dir+'rf_best.joblib') 

Parameters of best model: {'bootstrap': False, 'class_weight': 'balanced', 'max_depth': 36, 'min_samples_leaf': 1, 'n_estimators': 15}
Cross validation scores using that model: [0.95714286 0.99310345 0.97931034 0.99310345 0.96503497]


['../Data/classifier_data/rf_best.joblib']

## Improving (?) cross-validation with pre-defined arrays of indices and repeated cross-validation

Note that this will take considerably (4x) longer than the above code assuming I stick with 5-fold cross-validation above and `n_repetitions=20` below.

In [13]:
###Pre-defining n separate train/test splits
n_repetitions = 20
train_frac = 0.5/0.6 ###This essentially defines the training and validation set sizes

###This works by selecting iloc's
listy = list(range(0, ml_df_train.shape[0]))
train_lists = []
val_lists = []
for i in range(n_repetitions):
    random.Random(42+i).shuffle(listy)
    train_lists.append(listy[:int(len(listy)*train_frac)]) ###Select from beginning of list up to cut-point
    val_lists.append(listy[int(len(listy)*train_frac):]) ###Select cut-point onwards
###Zip these two together (and make sure I did it correctly!)
cv_splits = list(zip(train_lists, val_lists))
print(len(cv_splits), len(cv_splits[0]), len(cv_splits[0][0]), len(cv_splits[0][1]))

20 2 528 106


In [14]:
###Perform grid search using all the same parameters as previously defined
rf_AJH = RandomForestClassifier()
rf_gs_AJH = GridSearchCV(rf_AJH, params_rf, scoring=scoring_fxn, cv=cv_splits) ###Provide cv with list

#Fit the model
rf_gs_AJH.fit(ml_df_train.values, training_labels.values) ###Run on the values

###Find the model with the highest minimum accuracy across all n_repetitions of cross-validation
listy = list(zip(*[rf_gs_AJH.cv_results_['split{}_test_score'.format(i)] for i in range(n_repetitions)]))
print(len(listy), len(listy[0]))
listy = list(zip(*[rf_gs_AJH.cv_results_['params'], listy]))
print(len(listy), len(listy[0]))
listy = sorted(listy, key=lambda x: min(x[1]))
print(len(listy), len(listy[0]))
print(listy[0])
print(listy[-1])
best_params = listy[-1][0]
rf_min_AJH = RandomForestClassifier(**best_params)
rf_min_AJH.fit(ml_df_train, training_labels)

joblib.dump(rf_min_AJH, base_dir+'rf_highMinAJH.joblib'); 

2432 20
2432 2
2432 2
({'bootstrap': True, 'class_weight': 'balanced_subsample', 'max_depth': 12, 'min_samples_leaf': 1, 'n_estimators': 10}, (0.9256198347107438, 0.9523809523809524, 0.99009900990099, 0.9661016949152542, 0.9739130434782608, 0.975609756097561, 0.9921259842519685, 0.9821428571428572, 1.0, 0.9661016949152542, 0.9557522123893805, 0.9714285714285714, 0.972972972972973, 0.9655172413793104, 0.9767441860465117, 0.970873786407767, 0.9908256880733944, 0.9836065573770492, 0.9672131147540983, 0.9649122807017544))
({'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 40, 'min_samples_leaf': 1, 'n_estimators': 80}, (0.959349593495935, 0.9606299212598426, 0.99009900990099, 0.9747899159663865, 0.9739130434782608, 0.975609756097561, 0.9921259842519685, 0.9911504424778761, 1.0, 0.983050847457627, 0.9642857142857142, 0.9714285714285714, 0.972972972972973, 0.9743589743589743, 0.9846153846153847, 0.9807692307692307, 0.9908256880733944, 0.991869918699187, 0.97520661157024

In [15]:
print(best_params)

{'bootstrap': False, 'class_weight': 'balanced_subsample', 'max_depth': 40, 'min_samples_leaf': 1, 'n_estimators': 80}
